# CS671 Deep Learning & Applications - Tutorial II

Date: 18 April 2023 | Instructor: Dr. Dileep A.D. | References: https://keras.io/

### CNN Basics

In [5]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Dropout, Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, save_model

##### Simple 10 Class Image Classification N/w using Keras Functional API

In [22]:
inp = Input((224, 224, 1))

### LAYER 1
l1 = Conv2D(32, (3, 3), activation='relu')(inp)
l1 = MaxPooling2D((3, 3))(l1)

### LAYER 2
l2 = Conv2D(64, (3, 3), activation='relu')(l1)
l2 = MaxPooling2D((3, 3))(l2)

### LAYER 3
l3 = Conv2D(128, (3, 3), activation='relu')(l2)
l3 = MaxPooling2D((3, 3))(l3)

### Flattening
flatten = Flatten()(l3)

### Fully Connected Layers

l4 = Dense(256, activation='relu')(l3)
output = Dense(10, activation='softmax')(l4)

In [26]:
model = Model(inp, output)

### Pre-trained Models: VGG16/ResNet50

Whole list:  https://keras.io/api/applications/

#### Direct Classification

In [30]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
model = VGG16(weights='imagenet')

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
print('Predicted:', decode_predictions(preds, top=1)[0])

#### Extracting Features

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
model = VGG16(weights='imagenet', include_top=False)

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
features = model.predict(x)

#### Extracting Intermediate Layer Features

In [ ]:
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

#### Finetuning for New Sets of Classes

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
base_model = VGG16(weights='imagenet')

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(...)
model.fit(...)

#### Custom Inputs

In [ ]:
input_tensor = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=True)

### Visualizing CNN and Feature Maps

In [ ]:
for layer in model.layers:
    if 'conv' in layer.name:
        filters, biases = layer.get_weights()
        print(layer.name, filters.shape)

##### Visualize kernels

In [ ]:
model = VGG16()

filters, biases = model.layers[1].get_weights()

f_min, f_max = filters.min(), filters.max() # Normalising
filters = (filters - f_min) / (f_max - f_min)

##### Feature Maps

In [ ]:
feature_maps = model.predict(img)

ix = 2 # ith number of feature in corresponding layer.
plt.imshow(feature_maps[0, :, :, ix-1], cmap='viridis')
plt.show()

### Visualizing Patches that Maximally Activate a Neuron, Visualizing Influence of Input Pixel (Guided Back Propagation/ GradCAM)

In [ ]:
import tensorflow as tf

# Define some input data and trainable variables
x = tf.constant(3.0)
w = tf.Variable(4.0)

# Define a function to compute y = w*x
def compute_y(x, w):
    return w*x

# Use GradientTape to compute gradients of y with respect to w
with tf.GradientTape() as tape:
    y = compute_y(x, w)

grads = tape.gradient(y, w)

# Print the gradients
print(grads.numpy()) # Output: 3.0